# Citation Search
```Code by Prof. Karl Scheffler```

In [1]:
'''
This cell loads all data and builds the citation graph.

 * Most of the code below is for

    * getting a zip file from an online source,
    * unzipping it, and
    * processing the resulting text file format to get the data we want.

   You don't have to worry about the details of how this works.

 * The result is a directed NetworkX graph with an edge from node A to
   node B meaning that node A cites node B as a reference.

 * We also construct a dictionary, mapping the id of an article to the
   list of unique words appearing in that article. Capitalization is
   removed from all words.
'''

import networkx as nx
from collections import defaultdict
from requests import get
from io import BytesIO
from zipfile import ZipFile

# Data from: https://www.aminer.cn/citation
request = get('https://lfs.aminer.cn/lab-datasets/citation/citation-network1.zip')
zip_file = ZipFile(BytesIO(request.content))
first_lines = 50

print('Loading graph data and printing first', first_lines, 'lines\n')

g = nx.DiGraph()
article_words = {}
with zip_file.open('outputacm.txt', 'r') as fp:
    entry_count = int(next(fp).strip())
    print(entry_count, 'entries')
    for expected_index in range(entry_count):
        lines = []
        while True:
            line = next(fp).decode('utf-8').strip()
            if first_lines > 0:
                print(line)
            first_lines -= 1
            if line == '':
                break
            lines.append(line)
        entry = {}
        line_counter = 0
        for key, prefix, multiple, mapping in [
            ('title', '#*', False, str), ('authors', '#@', False, str),
            ('year', '#t', False, int), ('venue', '#c', False, str),
            ('id', '#index', False, int), ('references', '#%', True, int),
            ('abstract', '#!', False, str),
        ]:
            if multiple:
                entry[key] = []
            while (
                line_counter < len(lines) and
                lines[line_counter].startswith(prefix)
            ):
                value = mapping(lines[line_counter][len(prefix):])
                if multiple:
                    entry[key].append(value)
                else:
                    if key in entry:
                        import pdb
                        pdb.set_trace()
                    assert key not in entry
                    entry[key] = value
                line_counter += 1
        g.add_node(entry['id'])
        g.nodes[entry['id']]['title'] = entry['title']
        for ref_id in entry['references']:
            g.add_edge(entry['id'], ref_id)
        article_words[entry['id']] = set(
            entry.get('abstract', '').strip().lower().split())

print()
print('----------')
print('Done!')
print('The citation network has', g.number_of_nodes(), 'nodes and', g.number_of_edges(), 'edges')

Loading graph data and printing first 50 lines

629814 entries
#*Automated Deduction in Geometry: 5th International Workshop, ADG 2004, Gainesville, FL, USA, September 16-18, 2004, Revised Papers (Lecture Notes in Computer ... / Lecture Notes in Artificial Intelligence)
#@Hoon Hong,Dongming Wang
#t2006
#c
#index0

#*A+ Certification Core Hardware (Text & Lab Manual)
#@Charles J. Brooks
#t2003
#c
#index1

#*Performance engineering in industry: current practices and adoption challenges
#@Ahmed E. Hassan,Parminder Flora
#t2007
#cProceedings of the 6th international workshop on Software and performance
#index2
#!This panel session discusses performance engineering practices in industry. Presentations in the session will explore the use of lightweight techniques and approaches in order to permit the cost effective and rapid adoption of performance modeling research by large industrial software systems.

#*Dude, You Can Do It! How to Build a Sweeet PC
#@Darrel Creacy,Carlito Vicencio
#t2005


In [2]:
'''
This cell builds an _inverted index_ by mapping all unique words in the whole
data set to the article ids in which they appear.
'''
word_counts = defaultdict(int)
inverted_index = defaultdict(list)
for article_id, words in article_words.items():
    for word in words:
        word_counts[word] += 1
        inverted_index[word].append(article_id)

print('There are', len(inverted_index), 'unique words')

There are 807190 unique words


In [3]:
for word in ['human', 'computer', 'artificial']:
    print()
    print('The word', word, 'appears in', len(inverted_index[word]), 'articles. The first 5 are')
    for article_id in inverted_index[word][:5]:
        print(' *', g.nodes[article_id]['title'])


The word human appears in 9254 articles. The first 5 are
 * Webbots, Spiders, and Screen Scrapers
 * A hybrid approach for fabrication of polymeric BIOMEMS devices
 * Metrics for evaluating human information interaction systems
 * Implicit Meshes for Effective Silhouette Handling
 * Rapid and brief communication: An algorithm for semi-supervised learning in image retrieval

The word computer appears in 23774 articles. The first 5 are
 * Dude, You Can Do It! How to Build a Sweeet PC
 * A control word model for detecting conflicts between microoperations
 * Design team composition for high level language computer architectures
 * CompTIA A+ Exam Cram (Exams 220-602, 220-603, 220-604) (Exam Cram)
 * The Advantage Series: Microsoft Office Word 2003, Brief Edition, 1 edition

The word artificial appears in 4277 articles. The first 5 are
 * Controlling simulation games through rule-based scenarios
 * Game Programming for Teens (Game Programming)
 * Emotion representation and physiology assi

In [4]:
'''
 * The search function below uses the inverted index to find the articles
   containing all keywords.

 * It then sorts the articles using the PageRank algorithm.
'''

def search(phrase):
    print('Searching for %r' % phrase)

    # Find abstracts containing all keywords
    keywords = phrase.strip().lower().split()
    result_set = set(inverted_index[keywords[0]])
    for word in keywords[1:]:
        result_set.intersection_update(inverted_index[word])

    # Map all article ids to the same weight (1) for PageRank
    result_set = dict.fromkeys(result_set, 1)
    print('Results (%i hits):' % len(result_set))

    # Compute PageRank using results, sorted from highest to lowest score
    pagerank = sorted(
        nx.pagerank_scipy(g, personalization=result_set, max_iter=30).items(),
        key=lambda x: x[1],
        reverse=True)

    # Print results
    top_score = pagerank[0][1]
    for i in range(10):
        print('%2i. %3.0f%% - %s' % (i + 1, 100 * pagerank[i][1] / top_score, g.nodes[pagerank[i][0]]['title']))

In [5]:
search('Monte Carlo')

Searching for 'Monte Carlo'
Results (1181 hits):
 1. 100% - Simulation Modeling and Analysis, 2nd edition
 2.  84% - Random number generation and quasi-Monte Carlo methods
 3.  82% - Stochastic finite elements: a spectral approach
 4.  74% - Monte Carlo Statistical Methods (Springer Texts in Statistics)
 5.  62% - Statistical Timing Analysis Considering Spatial Correlations using a Single Pert-Like Traversal
 6.  56% - Monte Carlo methods. Vol. 1: basics
 7.  52% - Time series: theory and methods
 8.  48% - Statistical analysis with missing data
 9.  48% - Large sample properties of simulations using latin hypercube sampling
10.  47% - Simulation Modeling and Analysis, 3rd edition


In [6]:
search('Bayesian Inference')

Searching for 'Bayesian Inference'
Results (263 hits):
 1. 100% - Probabilistic reasoning in intelligent systems: networks of plausible inference
 2.  29% - An Introduction to Variational Methods for Graphical Models
 3.  28% - Information Theory, Inference & Learning Algorithms
 4.  25% - Gaussian Processes for Machine Learning (Adaptive Computation and Machine Learning)
 5.  22% - mStruct: a new admixture model for inference of population structure in light of both genetic admixing and allele mutations
 6.  22% - A general approach to heteroscedastic linear regression
 7.  21% - Transformation and weighting in regression
 8.  18% - Assessing Approximate Inference for Binary Gaussian Process Classification
 9.  17% - Naive Bayes models for probability estimation
10.  17% - Bayesian Inference and Optimal Design for the Sparse Linear Model
